## Libraries

In [1]:
import os, sys

sys.path.append(os.path.join(os.getcwd(), ".."))

In [2]:
import json
from concurrent.futures import ThreadPoolExecutor
from typing import List, Union, Optional
from tqdm import tqdm

from src.llm import BaseLLM, GroqLLM
from src.llm.utils import parse_llm_json_output
from src.utils import update_env_variable

## Translate

In [3]:
SYSTEM_PROMPT = """\
You are an AI language model acting as a Translator. Your task is to translate medical documents from English to Vietnamese. \
The goal of your translation is to be clear, concise, and easy to understand, while preserving the original meaning and tone.

You will receive input in JSON format, where only the values need to be translated, and the keys should remain unchanged. \
The output must also be in JSON format, retaining the original structure.
Example:
- Input:
```json
[
    {{  
        "patient_name": "John Doe",  
        "diagnosis": "Type 2 Diabetes",  
        "prescription": "Take 1 tablet of Metformin 500mg twice daily.",  
        "notes": "Patient should follow a low-carb diet and monitor blood sugar levels regularly."  
    }},
    {{
        <example 2>
    }}
]
```
- Output:
```json
[
    {{  
        "patient_name": "John Doe",  
        "diagnosis": "Bệnh tiểu đường loại 2",  
        "prescription": "Uống 1 viên Metformin 500mg hai lần mỗi ngày.",  
        "notes": "Bệnh nhân nên tuân theo chế độ ăn ít carbohydrate và theo dõi đường huyết thường xuyên."  
    }},
    {{
        <translated example 2>
    }}
]
```
---
Follow these step, translate these following input:
"""

In [4]:
def translate(
    llm: BaseLLM,
    texts: Union[Union[str, dict], List[Union[str, dict]]],
    system_prompt: Optional[str] = SYSTEM_PROMPT,
    batch_size: int = 1,
    max_workers: int = 3
) -> List[Union[str, dict]]:
    def wrapper(texts: List[Union[str, dict]]) -> str:
        update_env_variable()
        messages = [
            {"role": "system", "content": system_prompt},
        ]

        return llm.invoke(
            query=str(texts),
            messages=messages
        )
    
    if isinstance(texts, (str, dict)):
        texts = [texts]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(wrapper, texts[i:i + batch_size]) for i in range(0, len(texts), batch_size)]
        success = []
        failed = []
        not_processed = []
        for future in tqdm(futures, total=len(futures)):
            try:
                result = future.result()
                try:
                    success.extend(parse_llm_json_output(result, bracket_type="square"))
                except Exception as e:
                    failed.append(result)
            except Exception as e:
                idx = futures.index(future) * batch_size
                not_processed.extend(texts[idx:idx + batch_size])

    return success, failed, not_processed

In [5]:
with open("../data/translation_samples/input.json", "r", encoding="utf-8") as f:
    texts = json.load(f)

In [6]:
success, failed, not_processed = translate(
    llm=GroqLLM(model="llama-3.1-70b-versatile"),
    texts=texts
)

100%|██████████| 4/4 [00:03<00:00,  1.05it/s]


In [7]:
success

[{'question': 'abetalipoproteimemia Xin chào, tôi muốn biết liệu có bất kỳ hỗ trợ nào cho những người mắc bệnh abetalipoproteinemia không? Tôi chưa được chẩn đoán nhưng đã thực hiện nhiều xét nghiệm cho thấy tôi đang mắc bệnh này, tôi muốn tìm hiểu cách chẩn đoán và quản lý bệnh, xin cảm ơn',
  'answer_abs_summ': 'Abetalipoproteimemia, còn được gọi là hội chứng Bassen-Kornzweig, được chẩn đoán bằng xét nghiệm máu để kiểm tra Apolipoprotein B, thiếu hụt vitamin, biến dạng của các tế bào hồng cầu, xét nghiệm máu toàn diện và cholesterol. Các xét nghiệm khác bao gồm điện cơ đồ, kiểm tra mắt, tốc độ dẫn truyền thần kinh, phân tích mẫu phân và xét nghiệm di truyền để phát hiện đột biến trong gen MTP. Liều lượng lớn vitamin tan trong chất béo (vitamin A, vitamin D, vitamin E và vitamin K) được sử dụng để điều trị. Bổ sung axit linoleic cũng được khuyến nghị. Một chuyên gia dinh dưỡng có thể tư vấn về thay đổi chế độ ăn để ngăn ngừa các vấn đề về dạ dày. Bạn cần hạn chế lượng chất béo nạp vào

In [8]:
with open("../data/translation_samples/output.json", "w", encoding="utf-8") as f:
    json.dump(success, f, ensure_ascii=False, indent=4)